In [1]:
import gym
import gym_chrome_dino

In [2]:
env = gym.make('ChromeDino-v0')

WebDriverException: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=123.0.6312.86)
Stacktrace:
	GetHandleVerifier [0x00DC4CC3+225091]
	(No symbol) [0x00CF4E11]
	(No symbol) [0x00B99A7A]
	(No symbol) [0x00B960BF]
	(No symbol) [0x00B8B13C]
	(No symbol) [0x00B8C1CF]
	(No symbol) [0x00B8B3A3]
	(No symbol) [0x00B8ADC3]
	(No symbol) [0x00B8ACF3]
	(No symbol) [0x00B893C3]
	(No symbol) [0x00B89A1B]
	(No symbol) [0x00B9BC78]
	(No symbol) [0x00C063D1]
	(No symbol) [0x00BEF55C]
	(No symbol) [0x00C05CB0]
	(No symbol) [0x00BEF2F6]
	(No symbol) [0x00BC79B9]
	(No symbol) [0x00BC879D]
	sqlite3_dbdata_init [0x01239A63+4064547]
	sqlite3_dbdata_init [0x0124106A+4094762]
	sqlite3_dbdata_init [0x0123B968+4072488]
	sqlite3_dbdata_init [0x00F3C9C9+930953]
	(No symbol) [0x00D007E4]
	(No symbol) [0x00CFAD08]
	(No symbol) [0x00CFAE31]
	(No symbol) [0x00CECAA0]
	BaseThreadInitThunk [0x7596FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77217C5E+286]
	RtlGetAppContainerNamedObjectPat